In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
import keras
import os
import matplotlib.pyplot as plt
from tensorflow.keras.utils import img_to_array
import numpy as np 
import pandas as pd 
import os
import re
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

In [2]:
base_directory = '/mnt/DATA/jas123/Downloads/img_super/archive/folder/'
hires_folder = os.path.join(base_directory, 'high res')
lowres_folder = os.path.join(base_directory, 'low res')

In [6]:
data = pd.read_csv("/mnt/DATA/jas123/Downloads/img_super/archive/folder/image_data.csv")
data['low_res'] = data['low_res'].apply(lambda x: os.path.join(lowres_folder,x))
data['high_res'] = data['high_res'].apply(lambda x: os.path.join(hires_folder,x))
data.head()

,low_res,high_res
0,/mnt/DATA/jas123/Downloads/img_super/archive/f...,/mnt/DATA/jas123/Downloads/img_super/archive/f...
1,/mnt/DATA/jas123/Downloads/img_super/archive/f...,/mnt/DATA/jas123/Downloads/img_super/archive/f...
2,/mnt/DATA/jas123/Downloads/img_super/archive/f...,/mnt/DATA/jas123/Downloads/img_super/archive/f...
3,/mnt/DATA/jas123/Downloads/img_super/archive/f...,/mnt/DATA/jas123/Downloads/img_super/archive/f...
4,/mnt/DATA/jas123/Downloads/img_super/archive/f...,/mnt/DATA/jas123/Downloads/img_super/archive/f...


In [22]:
batch_size = 4

image_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.15)
mask_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.15)

train_hiresimage_generator = image_datagen.flow_from_dataframe(
        data,
        x_col='high_res',
        target_size=(512, 512),
        class_mode = None,
        batch_size = batch_size,
        seed=42,
        subset='training')

train_lowresimage_generator = image_datagen.flow_from_dataframe(
        data,
        x_col='low_res',
        target_size=(128, 128),
        class_mode = None,
        batch_size = batch_size,
        seed=42,
        subset='training')

val_hiresimage_generator = image_datagen.flow_from_dataframe(
        data,
        x_col='high_res',
        target_size=(512, 512),
        class_mode = None,
        batch_size = batch_size,
        seed=42,
        subset='validation')

val_lowresimage_generator = image_datagen.flow_from_dataframe(
        data,
        x_col='low_res',
        target_size=(128, 128),
        class_mode = None,
        batch_size = batch_size,
        seed=42,
        subset='validation')

train_generator = zip(train_lowresimage_generator, train_hiresimage_generator)
val_generator = zip(val_lowresimage_generator, val_hiresimage_generator)

def imageGenerator(train_generator):
    for (low_res, hi_res) in train_generator:
            yield (low_res, hi_res)

Found 3198 validated image filenames.
Found 3198 validated image filenames.
Found 564 validated image filenames.
Found 564 validated image filenames.


In [3]:
HSIZE =256
LSIZE =64

In [9]:
LSIZE = 128
HSIZE = 512
from keras import layers
import visualkeras
def down(filters , kernel_low_size, apply_batch_normalization = True):
    downsample = tf.keras.models.Sequential()
    downsample.add(layers.Conv2D(filters,kernel_low_size,padding = 'same', strides = 2))
    if apply_batch_normalization:
        downsample.add(layers.BatchNormalization())
    downsample.add(keras.layers.LeakyReLU())
    return downsample


def up(filters, kernel_low_size, dropout = False):
    upsample = tf.keras.models.Sequential()
    upsample.add(layers.Conv2DTranspose(filters, kernel_low_size,padding = 'same', strides = 2))
    if dropout:
        upsample.dropout(0.2)
    upsample.add(keras.layers.LeakyReLU())
    return upsample

def model():
    inputs = layers.Input(shape= [LSIZE,LSIZE,3])
    print(inputs.shape)
    d1 = down(128,(3,3),False)(inputs)
    print(d1.shape)
    d2 = down(128,(3,3),False)(d1)
    print(d2.shape)
    d3 = down(256,(3,3),True)(d2)
    print(d3.shape)
    d4 = down(512,(3,3),True)(d3)
    print(d4.shape)
    
    d5 = down(512,(3,3),True)(d4)
    print(d5.shape)
    #upsampling
    u1 = up(512,(3,3),False)(d5)
    print(u1.shape)
    u1 = layers.concatenate([u1,d4])
    u2 = up(256,(3,3),False)(u1)
    print(u2.shape)
    u2 = layers.concatenate([u2,d3])
    u3 = up(128,(3,3),False)(u2)
    print(u3.shape)
    u3 = layers.concatenate([u3,d2])
    u4 = up(128,(3,3),False)(u3)
    u4 = layers.concatenate([u4,d1])
    u5 = up(3,(3,3),False)(u4)
    u6 = up(3,(3,3),False)(u5)
    u7 = up(3,(3,3),False)(u6)
    output = layers.Conv2D(3,(2,2),strides = 1, padding = 'same')(u7)
    return tf.keras.Model(inputs=inputs, outputs=output)

model = model()
visualkeras.layered_view(model, draw_volume=False, legend=True, to_file='output1.png').show()
model.summary()
model.save('model.keras')

(None, 128, 128, 3)
(None, 64, 64, 128)
(None, 32, 32, 128)
(None, 16, 16, 256)
(None, 8, 8, 512)
(None, 4, 4, 512)
(None, 8, 8, 512)
(None, 16, 16, 256)
(None, 32, 32, 128)
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 sequential_36 (Sequential)  (None, 64, 64, 128)          3584      ['input_5[0][0]']             
                                                                                                  
 sequential_37 (Sequential)  (None, 32, 32, 128)          147584    ['sequential_36[0][0]']       
                                                                                                  
 sequential_38 (S

In [24]:
train_samples = train_hiresimage_generator.samples
val_samples = val_hiresimage_generator.samples

train_img_gen = imageGenerator(train_generator)
val_image_gen = imageGenerator(val_generator)

In [25]:
batch_size=1
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'mean_absolute_error',
              metrics = ['acc'])
H = model.fit(train_img_gen,
                    steps_per_epoch=train_samples//batch_size,
                    validation_data=val_image_gen,
                    validation_steps=val_samples//batch_size,
                    epochs=20)


Epoch 1/20
3198/3198 [==============================] - 437s 136ms/step - loss: 0.0573 - acc: 0.6502 - val_loss: 0.0407 - val_acc: 0.7619
Epoch 2/20
3198/3198 [==============================] - 391s 122ms/step - loss: 0.0425 - acc: 0.7433 - val_loss: 0.0385 - val_acc: 0.7506
Epoch 3/20
3198/3198 [==============================] - 391s 122ms/step - loss: 0.0412 - acc: 0.7482 - val_loss: 0.0410 - val_acc: 0.7368
Epoch 4/20
3198/3198 [==============================] - 389s 122ms/step - loss: 0.0403 - acc: 0.7509 - val_loss: 0.0375 - val_acc: 0.7589
Epoch 5/20
3198/3198 [==============================] - 391s 122ms/step - loss: 0.0394 - acc: 0.7517 - val_loss: 0.0394 - val_acc: 0.7505
Epoch 6/20
3198/3198 [==============================] - 391s 122ms/step - loss: 0.0388 - acc: 0.7527 - val_loss: 0.0368 - val_acc: 0.7656
Epoch 7/20
3198/3198 [==============================] - 390s 122ms/step - loss: 0.0381 - acc: 0.7535 - val_loss: 0.0369 - val_acc: 0.7619
Epoch 8/20
3198/3198 [============

In [ ]:
n = 0
for i,m in val_generator:
    img,mask = i,m
    sr1 = model.predict(img)
    if n < 20:
        fig, axs = plt.subplots(1 , 3, figsize=(20,4))
        axs[0].imshow(img[0])
        axs[0].set_title('Low Resolution Image')
        axs[1].imshow(mask[0])
        axs[1].set_title('High Resolution Image')
        axs[2].imshow(sr1[0])
        axs[2].set_title('Predicted High Resolution Image')
        plt.show()
        n+=1
    else:
        break